# Generativne mreže

Rekurentne neuronske mreže (RNN-ovi) i njihove varijante s kontroliranim ćelijama, poput ćelija s dugoročnim i kratkoročnim pamćenjem (LSTM-ovi) i jedinica s kontroliranim povratnim signalom (GRU-ovi), omogućile su modeliranje jezika, tj. mogu naučiti redoslijed riječi i pružiti predviđanja za sljedeću riječ u nizu. To nam omogućuje korištenje RNN-ova za **generativne zadatke**, poput običnog generiranja teksta, strojnog prevođenja, pa čak i opisivanja slika.

U RNN arhitekturi koju smo raspravili u prethodnoj jedinici, svaka RNN jedinica proizvodila je sljedeće skriveno stanje kao izlaz. Međutim, možemo dodati i drugi izlaz svakoj rekurentnoj jedinici, što bi nam omogućilo da dobijemo **niz** (koji je jednake duljine kao i izvorni niz). Štoviše, možemo koristiti RNN jedinice koje ne primaju ulaz na svakom koraku, već samo uzimaju neki početni vektor stanja i zatim proizvode niz izlaza.

U ovom ćemo bilježniku fokus staviti na jednostavne generativne modele koji nam pomažu generirati tekst. Radi jednostavnosti, izgradit ćemo **mrežu na razini znakova**, koja generira tekst slovo po slovo. Tijekom treniranja, potrebno je uzeti neki korpus teksta i podijeliti ga na nizove slova.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Izgradnja vokabulara znakova

Za izgradnju generativne mreže na razini znakova, potrebno je podijeliti tekst na pojedinačne znakove umjesto na riječi. Ovo se može postići definiranjem drugačijeg tokenizatora:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Pogledajmo primjer kako možemo kodirati tekst iz našeg skupa podataka:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Treniranje generativne RNN

Način na koji ćemo trenirati RNN za generiranje teksta je sljedeći. U svakom koraku uzet ćemo niz znakova duljine `nchars` i tražiti od mreže da generira sljedeći izlazni znak za svaki ulazni znak:

![Slika koja prikazuje primjer RNN generiranja riječi 'HELLO'.](../../../../../translated_images/hr/rnn-generate.56c54afb52f9781d.webp)

Ovisno o stvarnom scenariju, možda ćemo htjeti uključiti i neke posebne znakove, poput *kraj-sekvence* `<eos>`. U našem slučaju, želimo trenirati mrežu za beskonačno generiranje teksta, stoga ćemo fiksirati veličinu svakog niza na `nchars` tokena. Posljedično, svaki primjer za treniranje sastojat će se od `nchars` ulaza i `nchars` izlaza (što je ulazni niz pomaknut za jedan simbol ulijevo). Minibatch će se sastojati od nekoliko takvih nizova.

Način na koji ćemo generirati minibatcheve je da uzmemo svaki tekst vijesti duljine `l` i iz njega generiramo sve moguće kombinacije ulaz-izlaz (bit će ih `l-nchars` takvih kombinacija). Te kombinacije činit će jedan minibatch, a veličina minibatcheva bit će različita u svakom koraku treniranja.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Sada ćemo definirati generator mrežu. Ona može biti bazirana na bilo kojoj rekurentnoj ćeliji o kojoj smo raspravljali u prethodnoj jedinici (jednostavna, LSTM ili GRU). U našem primjeru koristit ćemo LSTM.

Budući da mreža prima znakove kao ulaz, a veličina vokabulara je prilično mala, ne trebamo sloj za ugradnju (embedding layer); ulaz kodiran u one-hot formatu može izravno ići u LSTM ćeliju. Međutim, budući da prosljeđujemo brojeve znakova kao ulaz, moramo ih kodirati u one-hot formatu prije nego ih proslijedimo u LSTM. To se postiže pozivanjem funkcije `one_hot` tijekom `forward` prolaza. Izlazni enkoder bit će linearni sloj koji će pretvoriti skriveno stanje u izlaz kodiran u one-hot formatu.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Tijekom treniranja želimo imati mogućnost uzorkovanja generiranog teksta. Da bismo to postigli, definirat ćemo funkciju `generate` koja će proizvesti izlazni niz duljine `size`, počevši od početnog niza `start`.

Način na koji to funkcionira je sljedeći. Prvo ćemo proslijediti cijeli početni niz kroz mrežu i uzeti izlazno stanje `s` i sljedeći predviđeni znak `out`. Budući da je `out` kodiran kao one-hot, uzimamo `argmax` kako bismo dobili indeks znaka `nc` u vokabularu, a zatim koristimo `itos` kako bismo odredili stvarni znak i dodali ga u rezultirajući popis znakova `chars`. Ovaj proces generiranja jednog znaka ponavlja se `size` puta kako bi se generirao potreban broj znakova.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Sad krenimo s treningom! Petlja za trening gotovo je ista kao u svim našim prethodnim primjerima, ali umjesto točnosti ispisujemo uzorkovani generirani tekst svakih 1000 epoha.

Posebnu pažnju treba obratiti na način na koji računamo gubitak. Moramo izračunati gubitak koristeći jednoznačno kodirani izlaz `out` i očekivani tekst `text_out`, koji je popis indeksa znakova. Srećom, funkcija `cross_entropy` očekuje nenormalizirani izlaz mreže kao prvi argument, a broj klase kao drugi, što je upravo ono što imamo. Također automatski provodi prosjek preko veličine minibatcha.

Trening također ograničavamo na `samples_to_train` uzoraka, kako ne bismo predugo čekali. Potičemo vas da eksperimentirate i pokušate s duljim treningom, moguće kroz nekoliko epoha (u tom slučaju trebali biste stvoriti dodatnu petlju oko ovog koda).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Ovaj primjer već generira prilično dobar tekst, ali može se dodatno poboljšati na nekoliko načina:
* **Bolja generacija minibatchova**. Način na koji smo pripremili podatke za treniranje bio je generiranje jednog minibatcha iz jednog uzorka. To nije idealno, jer minibatchevi imaju različite veličine, a neki se čak ne mogu generirati jer je tekst manji od `nchars`. Osim toga, mali minibatchevi ne iskorištavaju GPU dovoljno učinkovito. Pametnije bi bilo uzeti jedan veliki dio teksta iz svih uzoraka, zatim generirati sve ulazno-izlazne parove, promiješati ih i generirati minibatcheve jednake veličine.
* **Višeslojni LSTM**. Ima smisla isprobati 2 ili 3 sloja LSTM ćelija. Kao što smo spomenuli u prethodnoj jedinici, svaki sloj LSTM-a izvlači određene obrasce iz teksta, a u slučaju generatora na razini znakova možemo očekivati da će niži LSTM sloj biti odgovoran za izvlačenje slogova, a viši slojevi - za riječi i kombinacije riječi. Ovo se jednostavno može implementirati prosljeđivanjem parametra broja slojeva konstruktoru LSTM-a.
* Također biste mogli eksperimentirati s **GRU jedinicama** i vidjeti koje daju bolje rezultate, kao i s **različitim veličinama skrivenih slojeva**. Prevelik skriveni sloj može dovesti do prekomjernog prilagođavanja (npr. mreža će naučiti točan tekst), dok premala veličina možda neće dati dobar rezultat.


## Generiranje mekog teksta i temperatura

U prethodnoj definiciji `generate`, uvijek smo uzimali znak s najvećom vjerojatnošću kao sljedeći znak u generiranom tekstu. To je često rezultiralo time da se tekst "vrtio" između istih sekvenci znakova iznova i iznova, kao u ovom primjeru:
```
today of the second the company and a second the company ...
```

Međutim, ako pogledamo raspodjelu vjerojatnosti za sljedeći znak, može se dogoditi da razlika između nekoliko najvećih vjerojatnosti nije velika, npr. jedan znak može imati vjerojatnost 0.2, drugi 0.19, itd. Na primjer, kada tražimo sljedeći znak u sekvenci '*play*', sljedeći znak može jednako dobro biti razmak ili **e** (kao u riječi *player*).

To nas dovodi do zaključka da nije uvijek "pravedno" odabrati znak s većom vjerojatnošću, jer odabir drugog po redu također može dovesti do smislenog teksta. Mudrije je **uzorkovati** znakove iz raspodjele vjerojatnosti koju daje izlaz mreže.

Ovo uzorkovanje može se provesti pomoću funkcije `multinomial` koja implementira tzv. **multinomialnu raspodjelu**. Funkcija koja implementira ovo **meko** generiranje teksta definirana je u nastavku:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Uveli smo još jedan parametar nazvan **temperature**, koji se koristi za označavanje koliko čvrsto trebamo slijediti najveću vjerojatnost. Ako je temperatura 1.0, radimo pošteno multinomijalno uzorkovanje, a kada temperatura ide prema beskonačnosti - sve vjerojatnosti postaju jednake i nasumično biramo sljedeći znak. U primjeru ispod možemo primijetiti da tekst postaje besmislen kada previše povećamo temperaturu, a nalikuje "cikliranom" teško generiranom tekstu kada se približi 0.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
